In [0]:
# !pip install openpyxl

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
import re
import numpy as np
from datetime import datetime
import warnings
from collections import OrderedDict
import os
import yaml
import pyspark.sql.functions as F
warnings.filterwarnings('ignore')

In [0]:
%run "./fetch_data"

In [0]:
%run "./get_hl_from_skuname"

In [0]:
# with open('/Workspace/Users/ruofei.shen@budweiserapac.com/2024/4.ws_FIFO_inv/config.yml', 'r', encoding='utf-8') as file:
#     params_dict = yaml.safe_load(file)

### params

In [0]:
with open('./config.yml', 'r', encoding='utf-8') as file:
    params_dict = yaml.safe_load(file)
ws_params_dict_inv = params_dict['ws_params_dict_inv']  # inv数据使用列名
ws_params_dict = params_dict['ws_params_dict']     # stw数据 列名
ws_params_dict_str = params_dict['ws_params_dict_str']   # str列名

data_start_month = '202101'
data_end_month = '202412'

In [0]:
# data_type = get_parameter('data_type', 't1')
# data_type = get_parameter('data_type', 't15')
# data_type = get_parameter('data_type', 't1_t15') # 挂在t1下面的t1.5的write off
# data_version = get_parameter('data_version', 'v2025010301')

versions = VersionControl(ws_type='t1_t15', version='v20250107', comment='')
data_type = versions.ws_type
data_version = versions.version
print(data_type, data_version)

the version has not been saved before
t1_t15 v20250107


### funcs

In [0]:
def make_monthly_horizontal_df(lst, name):
    # print(len(lst))
    df = pd.DataFrame(lst, columns=[name]).T
    # df.columns = [x + 1 for x in df.columns]
    # df = df.add_prefix('month_')
    df.columns = month_name_index_lst
    return df


def get_stw_monthly_balance(begin_inv, stw: list, out_str: list, inv_expire_month=6, simu_months=12):
    """
    begin_inv:202212期末库存=2023年1月期初库存
    stw: 2023年12个月的stw进货量
    out_str:2023年12个月的出货量
    simu_months: 模拟的月份数，默认是1年即12个月
    result: 期初库存每个月的结余
    """
#     # 测试数据
#     begin_inv = 80
#     stw = [80]*2 + [100]*10     # 进货
#     out_str = [80]*7 + [80]*5
    check_month = 2
    begin_inv_remain = begin_inv
    stw_remain = stw.copy()             #  截止到当前月每个月的进货量还剩多少                
    stw_remain_monthly = []             # 每个月的结果合集
    begin_inv_remain_monthly = []       # 每个月的结果合集
    #     begin_inv_remain_lst = [None] * 12
    #     stw_remain_list = []        # 每个月进货stw的剩余
    str_remain_list = []
    for i in range(simu_months):
        stw_remain_list = []
        # print(f"1. month {i}: stw remain: {stw_remain}")
        if (out_str[i] <= begin_inv_remain) and (i <= (inv_expire_month - 1)) and (out_str[i] > 0):     # 期初库存足够扣
            begin_inv_remain -= out_str[i]  # 期初（年初）库存的剩余
            str_remain = 0
            str_remain_list.append(str_remain)
            stw_remain_list = stw_remain.copy()     # 模拟第i个月的时候 每个月进来的stw的剩余
            stw_remain_list = [stw_remain_list[stw_month] if stw_month <= i else None for stw_month in range(simu_months) ]    # 当前月后面的stw剩余置为空值
        else: # 期初库存不变
            if (begin_inv_remain > 0) and (i <= (inv_expire_month - 1)) and (out_str[i] > 0):
                # 期初库存大于零，但是不够第i个月STR用的 
                str_remain = out_str[i] - begin_inv_remain     # 这个月还欠多少货
                begin_inv_remain = 0
            else: # 期初库存是0或者已过期，或者 月i STR 是空的
                str_remain = out_str[i]
                # print(f"2. stw库存循环前的 str remain {str_remain}")
            
            stw_remain_list = stw_remain.copy()     # 模拟第i个月的时候 每个月进来的stw的剩余
            for j in range(len(stw_remain)):
                if j > i:                           # 第i个月不能用超过第i个月的进货
                    break
                if (i > (inv_expire_month + j)):  
                    pass                             # 如果到了8月，1月进货的stw已经不能用了
                else:                  
                    if stw_remain[j] > 0:
                        delta = min(str_remain, stw_remain[j])   # 这个月的剩余需求和当前月的进货余量
                        # print(f"3. stw_{j}: delta of str remain, stw_j {delta}")
                    else:            # 如果stw是负数，就都不扣了
                        delta = 0         
                    str_remain -= delta
                    # print(f"4.str remain after stw_{j}: {str_remain}")
                    try:
                        stw_remain[j] -= delta
                    except:
                        print(stw_remain[j], delta)
                # if j <= i:
                #     stw_remain_list.append(stw_remain[j])    #  大于当前月的进货不能用
                # else:
                #     stw_remain_list.append(None)
                stw_remain_list[j] = stw_remain[j]
                # print(f"5. stw remain after month{i} {stw_remain}") 
                if j == i:
                    str_remain_list.append(str_remain)       # 用于检验这个月str需求是否全被满足了
            stw_remain_list = [stw_remain_list[t] if t <= i else None for t in range(len(stw_remain_list))]
        stw_remain_monthly.append(stw_remain_list)
        begin_inv_remain_monthly.append(begin_inv_remain)
    #     return stw_remain_monthly, begin_inv_remain_monthly, str_remain_list
    stw_replenish_df = make_monthly_horizontal_df(stw, 'STW(+)')
    str_remain_df = make_monthly_horizontal_df(str_remain_list, 'str_remain')
    out_str_df = make_monthly_horizontal_df(out_str, 'STR(-)')
    begin_inv_remain_df = make_monthly_horizontal_df(begin_inv_remain_monthly, 'openning_balance')
    # stw_balance_df = pd.DataFrame(stw_remain_monthly, index=range(1, simu_months + 1), columns=range(1, simu_months + 1)).T  
    # stw_balance_df = stw_balance_df.add_prefix('month_')
    stw_balance_df = pd.DataFrame(stw_remain_monthly, index=month_name_index_lst, columns=month_name_index_lst).T 
    stw_balance_df.index = stw_balance_df.index.map(lambda x: str(x) + '_STW_balance')
    output = pd.concat([stw_replenish_df, out_str_df, begin_inv_remain_df, stw_balance_df, str_remain_df])
    year_begin_inv_balance = pd.DataFrame([begin_inv] + [None]*(len(output)-1), index=output.index, columns=['Openning Balance'])
    year_begin_inv_balance.index.name = 'index'
    return pd.concat([year_begin_inv_balance, output], axis=1)

In [0]:
def get_inv_simu(a, simu_months):
    ws_code = a.index[0][0]
    brand = a.index[0][1]
    brand_expire_month = brand_expire_month_dict.get(brand)
    a = pd.merge(stw_yr_month_index, a.reset_index(), how='left')
    a['payercode'] = a['payercode'].backfill()
    a['payercode'] = a['payercode'].ffill()
    a['brand_family_comdb'] = a['brand_family_comdb'].ffill()
    a['brand_family_comdb'] = a['brand_family_comdb'].backfill()
    a[ws_params_dict_str['agg_col']] = a[ws_params_dict_str['agg_col']].fillna(0)
    a[ws_params_dict['agg_col']] = a[ws_params_dict['agg_col']].fillna(0)
    stw_list = a[ws_params_dict['agg_col']].tolist()
    str_list = a[ws_params_dict_str['agg_col']].tolist()
    try:
        ws_begin_inv = begin_inv_by_ws_by_brand.loc[(ws_code, brand)]
    except:
        ws_begin_inv = 0
    result_df = get_stw_monthly_balance(ws_begin_inv, stw_list, str_list, inv_expire_month=brand_expire_month, simu_months=simu_months)
    result_df = result_df
    return result_df

### 读数

In [0]:
def get_sku_hl_mapping_table():
    sku_hl_mapping = spark.sql(
    """
    WITH CIO AS (
    SELECT
        DISTINCT cio_id AS cio_code,
        c_product as cio_name,
        regexp_replace(capacity, '([a-zA-Z])', '') / 1000.0 as container_L,
        -- trim(C2.Seg) AS Segment,
        packagesize
    FROM
        bees_products_ods.product_center_cio_sku C1
        LEFT JOIN bees_products_ods.product_center_cio_mapping C2 ON C2.cio_code = C1.cio_id
    WHERE
        deleted = 0
        AND regexp_replace(capacity, '([a-zA-Z])', '') > 0
        AND C2.Seg IS NOT NULL
        AND COALESCE(packagesize, 0) > 0
    )
    SELECT
    DISTINCT SC.sap_sku_code as SKUCode, K1.cio_code
    ,K1.cio_name
    ,K1.container_L * K1.packagesize/100 HL
    FROM
    CIO K1
    JOIN bees_products_ods.product_center_ref_sap_to_cio SC ON SC.cio_sku_code = K1.cio_code
    """
    ).toPandas()
    return sku_hl_mapping

def get_sku_sap_wccs_code_mapping_table():
    wccs_productid_sap_sku_mapping = spark.sql(
    """
    select distinct product_id, cast(sap_skudm as int) as SKUCode from wccs_ods.bas_material
    """).toPandas()
    return wccs_productid_sap_sku_mapping     

In [0]:
used_wccs_sku_list, sku_filtered, product_id_mapping = get_used_brand_family() # sku_filtered是过滤了四个brand的sku mapping表；product_id_mapping是全量的sku hl mapping表

In [0]:
begin_inv = get_inv_data(used_wccs_sku_list, sku_filtered, used_qty_col='期初库存')

没对应上SKUCode/HL的wccs product id ['P02816' 'P02747' 'P02746' 'P02931' 'P03012' 'P03002' 'P02896' 'P02820'
 'P02650' 'P02843' 'P02708' 'P03001' 'P03071' 'P03020' 'P00012' 'P00007'
 'P01040' 'P00024' 'P00043' 'P00033' 'P00038' 'P00040' 'P00032' 'P00028'
 'P00020' 'P00025' 'P00015' 'P00034' 'P00036' 'P00031' 'P00029' 'P00019'
 'P00014' 'P00027' 'P00035' 'P00830' 'P00211' 'P00022' 'P00030' 'P00627'
 'P02935' 'P03006' 'P02757' 'P00315' 'P00047' 'P00045' 'P00076' 'P00046'
 'P00055' 'P00098' 'P00053' 'P00051' 'P00050' 'P00064' 'P00062' 'P00058'
 'P00059' 'P01018' 'P00061' 'P00063' 'P00867' 'P00078' 'P02791' 'P02799'
 'P00073' 'P00074' 'P00056' 'P00065' 'P00067' 'P00072' 'P00057' 'P00233'
 'P00408' 'P00329' 'P00307' 'P00267' 'P00314' 'P00272' 'P00317' 'P00359'
 'P00444' 'P00481' 'P00343' 'P00704' 'P00249' 'P00301' 'P00254' 'P00260'
 'P00327' 'P00273' 'P00278' 'P00446' 'P00299' 'P00316' 'P00225' 'P00633'
 'P00401' 'P00631' 'P00469' 'P00295' 'P00653' 'P00558' 'P00231' 'P00854'
 'P00585' 'P00308' '

In [0]:
if data_type in ['t1', 't15']:
    stw = get_stw_data(start=data_start_month, end=data_end_month)
if data_type == 't1_t15':
    stw = get_t15_stw_monthly_from_t1(product_id_mapping, start=data_start_month, end=data_end_month)

202101 202412
     t15_code   t1_code  ...  SKUCode  HL
443  30011577  30006208  ...  35211.0 NaN

[1 rows x 12 columns]


In [0]:
# details = spark.read.parquet('/mnt/srf/inv/fifo_inventory_details_month12_t15_to_poc_write_off_v2025010302')
# details.filter(F.expr('t15_code == "30011295" and brand_family_comdb == "HBO"')).display()

payercode,brand_family_comdb,expire_month,ws_name,bu,region,index,Openning Balance,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202401,202402,202403,202404,202405,202406,202407,202408,202409,202410,202411,202412,t1_code,t15_code
30015993_30011295,HBO,12,null,东北|BU_NE,黑龙江东区|Region_Heilongjiang_East,STW(+),2527.8348,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30015993,30011295
30015993_30011295,HBO,12,null,东北|BU_NE,黑龙江东区|Region_Heilongjiang_East,STR(-),null,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30015993,30011295
30015993_30011295,HBO,12,null,东北|BU_NE,黑龙江东区|Region_Heilongjiang_East,openning_balance,null,2527.8348,2527.8348,2527.8348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,2526.3348,30015993,30011295
30015993_30011295,HBO,12,null,东北|BU_NE,黑龙江东区|Region_Heilongjiang_East,202101_STW_balance,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30015993,30011295
30015993_30011295,HBO,12,null,东北|BU_NE,黑龙江东区|Region_Heilongjiang_East,202102_STW_balance,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30015993,30011295
30015993_30011295,HBO,12,null,东北|BU_NE,黑龙江东区|Region_Heilongjiang_East,202103_STW_balance,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30015993,30011295
30015993_30011295,HBO,12,null,东北|BU_NE,黑龙江东区|Region_Heilongjiang_East,202104_STW_balance,null,null,null,null,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,30015993,30011295
30015993_30011295,HBO,12,null,东北|BU_NE,黑龙江东区|Region_Heilongjiang_East,202105_STW_balance,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30015993,30011295
30015993_30011295,HBO,12,null,东北|BU_NE,黑龙江东区|Region_Heilongjiang_East,202106_STW_balance,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30015993,30011295
30015993_30011295,HBO,12,null,东北|BU_NE,黑龙江东区|Region_Heilongjiang_East,202107_STW_balance,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30015993,30011295


In [0]:
# stw.query('(payercode==30015707) & (brand_family_comdb=="HKOW")&(billingyearmonth <= "202410")')['stw_hl'].sum()

In [0]:
if data_type == "t1":     # t1 to all
    t1_to_all_str = spark.sql(f"select * from finance_ds_inventory_dmt.finance_poc_datahub_t1ws_wccs_str")
    t1_to_all_str_agg = t1_to_all_str.groupby(['payercode', 'payer_name', 'payer_bu_code', 'payer_bu_name', 'payer_region_code', 'payer_region_name','month', 'Brand_Family_ComDB','skucode']).agg(F.sum('str_qty').alias('str_qty'))
    ws_str = get_str_data(product_id_mapping, t1_to_all_str_agg, start=data_start_month, end=data_end_month)
    print(ws_str.shape)

elif data_type == "t15":   # t1.5->poc STR
    t1_5_to_poc_db_name = 'finance_ds_inventory_dmt.finance_poc_datahub_mid_t1ws_wccs_str' # T1.5卖给每个POC的
    ws_str = get_str_data(product_id_mapping, t1_5_to_poc_db_name, start=data_start_month, end=data_end_month)
    print(ws_str.shape)

elif data_type == 't1_t15':    # 挂在t1下面的t1.5的
    ws_str = get_t15_str_monthly(product_id_mapping, start=data_start_month, end=data_end_month)
    print(ws_str.shape)

else:
    raise TypeError('should be t1 or t15 or t1_t15')

202101 202412
(36639, 13)


In [0]:
ws_attributes = ws_str[['payercode', 'payer_bu_name', 'payer_region_name']].drop_duplicates(subset=['payercode'])
ws_attributes2 = stw[['payercode', 'payer_bu_name', 'payer_region_name']].drop_duplicates(subset=['payercode'])
if data_type != 't1_t15':
    ws_attributes['payercode'] = ws_attributes['payercode'].astype(int)
    ws_attributes2['payercode'] = ws_attributes2['payercode'].astype(int)
    ws_str['payercode'] = ws_str['payercode'].astype(int)

ws_attributes = ws_attributes.set_index('payercode')
ws_attributes2 = ws_attributes2.set_index('payercode')

ws_bu_dict = dict(ws_attributes['payer_bu_name'])
ws_bu_dict2 = dict(ws_attributes2['payer_bu_name'])
ws_bu_dict.update(ws_bu_dict2)
ws_region_dict = dict(ws_attributes['payer_region_name'])
ws_region_dict2 = dict(ws_attributes2['payer_region_name'])
ws_region_dict.update(ws_region_dict2)

ws_code_name_payercode_df = get_ws_name_payercode_mapping()
ws_name_dict = dict(ws_code_name_payercode_df['payername'])

### 数据聚合

In [0]:
ws_params_dict_inv['agg_col'] = 'inv_hl'
ws_params_dict['agg_col'] = 'stw_hl'
ws_params_dict_str['agg_col'] = 'str_hl'

inv_expire_month = 12    # 库存9个月后失效，之前是6个月
brand_expire_month_dict = {'BUD': inv_expire_month, 'HBI': inv_expire_month, 'HBO': inv_expire_month, 'HKOW':inv_expire_month, 'ISP': inv_expire_month}

In [0]:
ws_code_mapping_table = spark.sql("select distinct wholesalerid as ws_code, int(Pay_to) as payercode  from bcc_baseline_ods.wccs_ws").toPandas()  # 一个w对着一个300；一个300对着多个W

if data_type in ['t1', 't15']:
    begin_inv2 = pd.merge(begin_inv, ws_code_mapping_table, left_on=ws_params_dict_inv['ws_id_name'], right_on='ws_code', how='left')
    begin_inv2['payercode'] = begin_inv2['payercode'].apply(lambda x: None if np.isnan(x) else int(x))
    # begin_inv.head() 这个是期初库存数据map上百升数和brandfamily的底表
elif data_type == 't1_t15':
    ws_code_mapping_table['payercode'] = ws_code_mapping_table['payercode'].apply(lambda x: None if np.isnan(x) else str(int(x)))
    begin_inv2 = pd.merge(begin_inv, ws_code_mapping_table, left_on=ws_params_dict_inv['ws_id_name'], right_on='ws_code', how='left')
    all_t1_t15_comb = get_all_t1_t15_comb()
    begin_inv2 = pd.merge(begin_inv2, all_t1_t15_comb, left_on='payercode', right_on='t15_code', how='left')
    begin_inv2['payercode']  = begin_inv2.apply(lambda x : cat_code(x), axis=1)
else:
    raise TypeError('should be t1 or t15 or t1_t15')

begin_inv_by_ws_by_brand = begin_inv2.groupby(['payercode', ws_params_dict_inv['brand_col']])[ws_params_dict_inv['agg_col']].sum()   # 期初库存by ws by brand

1883


In [0]:
# begin_inv_by_ws_by_brand[(30015707, 'HKOW')]

In [0]:
stw[ws_params_dict_str['month_col']] = stw[ws_params_dict['month_col']]
stw_monthly_by_ws_by_brand = stw.groupby([ws_params_dict['ws_id_name'], ws_params_dict['brand_col'], ws_params_dict_str['month_col']])[ws_params_dict['agg_col']].sum().astype(float)

str_monthly_by_ws_by_brand = ws_str.groupby([ws_params_dict_str['ws_id_name'], ws_params_dict_str['brand_col'], ws_params_dict_str['month_col']])[ws_params_dict_str['agg_col']].sum().astype(float)

ws_brand_stw_str = pd.merge(stw_monthly_by_ws_by_brand, str_monthly_by_ws_by_brand, left_index=True, right_index=True, how='outer')
ws_brand_stw_str.reset_index().display()

In [0]:
# ws_brand_stw_str.loc[(30015707, 'HKOW', slice(None))]['stw_hl'][:-2].sum()

In [0]:
vivian_ws_exclude1 = pd.read_excel("/dbfs/FileStore/tables/srf/fifo_inv_simu/src_data/exclude_WS.xlsx", sheet_name='KAEC')
vivian_ws_exclude2 = pd.read_excel("/dbfs/FileStore/tables/srf/fifo_inv_simu/src_data/exclude_WS.xlsx", sheet_name='太古')
exclude_ws1 = set(vivian_ws_exclude1['付款方代码'].unique())
exclude_ws2 = set(vivian_ws_exclude2['payercode'].unique())
exclude_ws = exclude_ws1.union(exclude_ws2)

# 是buse但不是湖南的ws list
region_ws_dict = defaultdict(list)
for ws_code, region in ws_region_dict.items():
    region_ws_dict[region].append(ws_code)

bu_ws_dict = defaultdict(list)
for ws_code, bu in ws_bu_dict.items():
    bu_ws_dict[bu].append(ws_code)

buse_ws = set(bu_ws_dict.get('东南区|BU_SE'))
hunan_ws = set(region_ws_dict.get('湖南|Region_Hunan'))
buse_non_hunan_ws = buse_ws - hunan_ws

ws_brand_stw_str2 = ws_brand_stw_str[~ws_brand_stw_str.index.get_level_values('payercode').isin(exclude_ws)]   # 1.去除KAEC和太古里的经销商

ws_brand_stw_str2 = ws_brand_stw_str2[~ws_brand_stw_str2.index.get_level_values('payercode').isin(buse_non_hunan_ws)] # 2. BUSE里只要湖南

In [0]:
# 区分有无str的ws
has_str_ws = ws_str['payercode'].unique()
ws_brand_stw_str2_has_str = ws_brand_stw_str2[ws_brand_stw_str2.index.get_level_values('payercode').isin(has_str_ws)]  # 有str的
ws_brand_stw_str2_no_str = ws_brand_stw_str2[~ws_brand_stw_str2.index.get_level_values('payercode').isin(has_str_ws)]  # 没str的

### main

In [0]:
month_cnt = len(stw['billingyearmonth'].unique())      # 2021.1到24年10月是46个月；每个经销商都要46个月
print(f"stw月份数：{month_cnt} 最大月: {stw['billingyearmonth'].max()}")
print(f"str月份数：{len(ws_str['month'].unique())} 最大月: {ws_str['month'].max()}")
stw_yr_month_index = stw['billingyearmonth'].drop_duplicates().sort_values()
stw_yr_month_index = stw_yr_month_index.rename('month')
# str_yr_month_index = ws_str['month'].drop_duplicates().sort_values()   # 长得是一样的
month_name_list = stw['billingyearmonth'].unique()
month_name_list.sort()
month_name_index_lst = [f"{month_name}" for month_name in month_name_list]

stw月份数：48 最大月: 202412
str月份数：48 最大月: 202412


In [0]:
# sample_data = ws_brand_stw_str.loc[(30018687, slice(None), slice(None)), :]

In [0]:
def get_simulation(ws_brand_stw_str):
    all_ws_result = ws_brand_stw_str.groupby(['payercode', 'brand_family_comdb']).apply(lambda df: get_inv_simu(df, simu_months=month_cnt)).reset_index('index')
    all_ws_result.insert(0, 'region', all_ws_result.index.get_level_values('payercode').map(ws_region_dict))
    all_ws_result.insert(0, 'bu', all_ws_result.index.get_level_values('payercode').map(ws_bu_dict))
    all_ws_result.insert(0, 'ws_name', all_ws_result.index.get_level_values('payercode').map(ws_name_dict))
    all_ws_result.insert(0, 'expire_month', all_ws_result.index.get_level_values('brand_family_comdb').map(brand_expire_month_dict))
    all_ws_result = all_ws_result.reset_index()

    if data_type == 't1_t15':
        all_ws_result[['t1_code', 't15_code']] = all_ws_result.apply(lambda x: split_paycode(x), axis=1,result_type='expand')
        all_ws_result['t15_code'] = all_ws_result['t15_code'].astype(int)
        all_ws_result['ws_name'] = all_ws_result['t15_code'].apply(lambda x: ws_name_dict.get(x))
    return all_ws_result


def save_parquet(all_ws_result, data_type, data_version):
    ws_brand_df_spark = spark.createDataFrame(all_ws_result)
    # ws_brand_df_spark.write.mode("overwrite").option('overwriteSchema',True).parquet(f'/mnt/srf/fifo_inventory_details_month{inv_expire_month}')
    if data_type == 't15':
        name = f'/mnt/srf/inv/fifo_inventory_details_month{inv_expire_month}_new_t15_poc_{data_version}'
    if data_type == 't1':
        name = f'/mnt/srf/inv/fifo_inventory_details_month{inv_expire_month}_t1_to_all_{data_version}'
        
    if data_type == 't1_t15':
        name = f'/mnt/srf/inv/fifo_inventory_details_month{inv_expire_month}_t15_to_poc_write_off_{data_version}'

    print(f"ws type: {data_type} path: {name}")
    ws_brand_df_spark.write.mode("overwrite").option('overwriteSchema', True).parquet(name)

In [0]:
all_ws_result = get_simulation(ws_brand_stw_str2_has_str)
versions.save_version(force=False)   # force=False检查版本是否已经存在，force=True直接覆盖
save_parquet(all_ws_result, data_type=data_type, data_version=data_version)

all_ws_result2 = get_simulation(ws_brand_stw_str2_no_str)
versions.save_version(ws_type=data_type, version=f'{data_version}_no_str', force=False)
save_parquet(all_ws_result2, data_type=data_type, data_version=f'{data_version}_no_str')

the version has not been saved before
t1_t15, v20250107, 
ws type: t1_t15 path: /mnt/srf/inv/fifo_inventory_details_month12_t15_to_poc_write_off_v20250107
the version has not been saved before
t1_t15, v20250107_no_str, 
ws type: t1_t15 path: /mnt/srf/inv/fifo_inventory_details_month12_t15_to_poc_write_off_v20250107_no_str


In [0]:
# if data_type == 't15':
#     table_name = f"finance_ds_inventory_dmt.inventory_details_{inv_expire_month}_new_t15_poc"
if data_type == 't1':
    parquet_path = f'/mnt/srf/inv/fifo_inventory_details_month{inv_expire_month}_t1_to_all_{data_version}'
    table_name = f"finance_ds_inventory_dmt.inventory_details_{inv_expire_month}_{data_type}_to_all_{data_version}"
if data_type == 't1_t15':
    parquet_path = f'/mnt/srf/inv/fifo_inventory_details_month{inv_expire_month}_t15_to_poc_write_off_{data_version}'
    table_name = f"finance_ds_inventory_dmt.inventory_details_{inv_expire_month}_{data_type}_{data_version}"

ws_brand_df_spark = spark.read.parquet(parquet_path)
ws_brand_df_spark = (ws_brand_df_spark
                     .withColumnRenamed('Openning Balance', 'Openning_Balance')
                     .withColumn('inv_expire_month', F.lit(inv_expire_month))
                     .withColumn('version', F.lit(data_version))
                     .withColumn('insert_time', F.lit(datetime.now()))
                     )
print(table_name)
ws_brand_df_spark.write.mode("overwrite").format("delta").option('overwriteSchema',True).saveAsTable(table_name)  #写入数据库

finance_ds_inventory_dmt.inventory_details_12_t1_t15_v20250107_no_str


In [0]:
%sh
ls /dbfs/mnt/srf

algo_selling
b2c
bc
boiler
consumer_tagging
dp
fifo_inventory_details_month12_new_t15_poc
fifo_inventory_details_month12_new_t15_poc_v2024122301
fifo_inventory_details_month12_new_t15_poc_v2024122602
fifo_inventory_details_month12_t15_to_poc_write_off_v2024122602
fifo_inventory_details_month12_t1_to_all
fifo_inventory_details_month12_t1_to_all_v2024122301
fifo_inventory_details_month12_t1_to_all_v2024122602
hackathon_sales
inv
poc_tag
smartpi


In [0]:
# dbutils.fs.rm ("mnt/srf/fifo_inventory_details_month12_t1_to_all",recurse=True)
# import re
# directory = "/mnt/srf" 
# files = dbutils.fs.ls(directory) 
# pattern = re.compile(r".*test.*")
# for file_info in files: 
#     if pattern.search(file_info.name):
#         dbutils.fs.rm(file_info.path, recurse=True)
#         print(f"Deleted {file_info.path}")

True

In [0]:
# dbutils.notebook.exit('end')